Netflix Recommendation System uygulamamız için modelimizi kaydedeceğiz

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
# Veriyi Yükle ve EDA

In [4]:
df = pd.read_csv("netflixData.csv")

In [5]:
df.head()

,Show Id,Title,Description,Director,Genres,Cast,Production Country,Release Date,Rating,Duration,Imdb Score,Content Type,Date Added
0,cc1b6ed9-cf9e-4057-8303-34577fb54477,(Un)Well,This docuseries takes a deep dive into the luc...,NaN,Reality TV,NaN,United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
1,e2ef4e91-fb25-42ab-b485-be8e3b23dedb,#Alive,"As a grisly virus rampages a city, a lone man ...",Cho Il,"Horror Movies, International Movies, Thrillers","Yoo Ah-in, Park Shin-hye",South Korea,2020.0,TV-MA,99 min,6.2/10,Movie,"September 8, 2020"
2,b01b73b7-81f6-47a7-86d8-acb63080d525,#AnneFrank - Parallel Stories,"Through her diary, Anne Frank's story is retol...","Sabina Fedeli, Anna Migotto","Documentaries, International Movies","Helen Mirren, Gengher Gatti",Italy,2019.0,TV-14,95 min,6.4/10,Movie,"July 1, 2020"
3,b6611af0-f53c-4a08-9ffa-9716dc57eb9c,#blackAF,Kenya Barris and his family navigate relations...,NaN,TV Comedies,"Kenya Barris, Rashida Jones, Iman Benson, Genn...",United States,2020.0,TV-MA,1 Season,6.6/10,TV Show,NaN
4,7f2d4170-bab8-4d75-adc2-197f7124c070,#cats_the_mewvie,This pawesome documentary explores how our fel...,Michael Margolis,"Documentaries, International Movies",NaN,Canada,2020.0,TV-14,90 min,5.1/10,Movie,"February 5, 2020"


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5967 entries, 0 to 5966
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Show Id             5967 non-null   object 
 1   Title               5967 non-null   object 
 2   Description         5967 non-null   object 
 3   Director            3903 non-null   object 
 4   Genres              5967 non-null   object 
 5   Cast                5437 non-null   object 
 6   Production Country  5408 non-null   object 
 7   Release Date        5964 non-null   float64
 8   Rating              5963 non-null   object 
 9   Duration            5964 non-null   object 
 10  Imdb Score          5359 non-null   object 
 11  Content Type        5967 non-null   object 
 12  Date Added          4632 non-null   object 
dtypes: float64(1), object(12)
memory usage: 606.2+ KB


In [7]:
df.isnull().sum()

Show Id                  0
Title                    0
Description              0
Director              2064
Genres                   0
Cast                   530
Production Country     559
Release Date             3
Rating                   4
Duration                 3
Imdb Score             608
Content Type             0
Date Added            1335
dtype: int64

In [8]:
# Veri Temizleme ve Ön İşleme

In [9]:
df['Description'] = df['Description'].fillna('')

In [10]:
# Eksik verileri dolduruyoruz

In [11]:
df['Genres'] = df['Genres'].fillna('')

In [12]:
df['Director'] = df['Director'].fillna('')

In [13]:
df['Cast'] = df['Cast'].fillna('')

In [14]:
# Metin temizleme fonksiyonu

In [15]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)
    return text

In [16]:
# Tüm önemli özellikleri birleştirip tek bir "Soup" (Çorba) oluşturuyoruz

In [17]:
df['combined_features'] = (
    df['Description'] + " " + 
    df['Genres'] + " " + 
    df['Director'] + " " + 
    df['Cast']
)

In [18]:
# Bu yöntem, hem konuya hem türe hem de oyuncuya göre benzerlik bulmamızı sağlar.

In [19]:
import re

In [20]:
df['combined_features'] = df['combined_features'].apply(clean_text)

In [21]:
# TF-IDF Vektörleştirme

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000) # İngilizce stop words (the, a, and vb.) çıkarılır

In [24]:
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [25]:
# Kosinüs Benzerliği (Cosine Similarity) Hesaplama

In [26]:
from sklearn.metrics.pairwise import linear_kernel

In [27]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # linear_kernel, cosine_similarity ile aynı işi yapar ama daha hızlıdır.

In [28]:
# Save

In [29]:
indices = pd.Series(df.index, index=df['Title']).drop_duplicates()

In [30]:
# Indeksleri kolay bulmak için bir harita oluşturduk

In [31]:
import joblib

In [32]:
joblib.dump(cosine_sim, "netflix_cosine_sim.pkl")

['netflix_cosine_sim.pkl']

In [33]:
joblib.dump(indices, "netflix_indices.pkl")

['netflix_indices.pkl']

In [34]:
df.to_csv("netflix_processed.csv", index=False)